In [1]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d gpiosenka/iconic-wwii-aircraft-imagesclassification

100%|██████████| 254M/254M [00:09<00:00, 28.9MB/s]


In [2]:
!unzip iconic-wwii-aircraft-imagesclassification.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/Douglas C-47 Skytrain/163.jpg  
  inflating: train/Douglas C-47 Skytrain/164.jpg  
  inflating: train/Douglas C-47 Skytrain/165.jpg  
  inflating: train/Douglas C-47 Skytrain/166.jpg  
  inflating: train/Douglas C-47 Skytrain/167.jpg  
  inflating: train/Douglas C-47 Skytrain/168.jpg  
  inflating: train/Douglas C-47 Skytrain/169.jpg  
  inflating: train/Douglas C-47 Skytrain/170.jpg  
  inflating: train/Douglas C-47 Skytrain/171.jpg  
  inflating: train/Douglas C-47 Skytrain/172.jpg  
  inflating: train/Douglas C-47 Skytrain/173.jpg  
  inflating: train/Douglas C-47 Skytrain/174.jpg  
  inflating: train/Douglas C-47 Skytrain/175.jpg  
  inflating: train/Douglas C-47 Skytrain/176.jpg  
  inflating: train/Douglas C-47 Skytrain/177.jpg  
  inflating: train/Douglas C-47 Skytrain/178.jpg  
  inflating: train/Douglas C-47 Skytrain/179.jpg  
  inflating: train/Douglas C-47 Skytrain/180.jpg  
  inflating: train/Douglas C-47

In [5]:
!ls

'airplanes-30-(200 X 200)-96.32.h5'		 sample_data
 airplanes.csv					 test
 drive						 train
 iconic-wwii-aircraft-imagesclassification.zip	 valid


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np 
import pandas as pd

In [21]:
train_gen = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    rescale=1./255,
    horizontal_flip=True
)

train_set = train_gen.flow_from_directory(
    "train/",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 7317 images belonging to 30 classes.


In [22]:
valid_gen = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    rescale=1./255,
    horizontal_flip=True
)

valid_set = valid_gen.flow_from_directory(
    "valid/",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 150 images belonging to 30 classes.


In [23]:
train_set.class_indices

{'Avro Lancaster': 0,
 'Bell P-63 Kingcobra': 1,
 'Boeing B-17 Flying Fortress': 2,
 'Boeing B-29 Superfortress': 3,
 'Boeing P-26 Peashooter': 4,
 'Brewster F2A Buffalo': 5,
 'Consolidated PBY Catalina': 6,
 'Curtiss P-40 Warhawk': 7,
 'Douglas A-20 Havoc': 8,
 'Douglas C-47 Skytrain': 9,
 'Douglas SBD Dauntless': 10,
 'Douglas TBD Devastator': 11,
 'Focke-Wulf Fw 190': 12,
 'Grumman F6F Hellcat': 13,
 'Grumman F7F Tigercat': 14,
 'Grumman TBF Avenger': 15,
 'Hawker Hurricane': 16,
 'Junkers Ju 87': 17,
 'Lockheed P-38 Lightning': 18,
 'Messerschmitt Bf 109': 19,
 'Mitsubishi A6M Zero': 20,
 'North American P51 Mustang': 21,
 'Northrop P-61 Black Widow': 22,
 'Republic P-43 Lancer': 23,
 'Republic P-47 Thunderbolt': 24,
 'Seversky P-35': 25,
 'Supermarine Spitfire': 26,
 'Vought F4U Corsair': 27,
 'Waco CG-4': 28,
 'de Havilland Mosquito': 29}

In [28]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, strides=2, input_shape=(224, 224, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(420, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(240, activation='relu'))
model.add(layers.Dense(30, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 111, 111, 32)      896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 55, 55, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 27, 27, 32)        9248      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 5408)              0         
                                                                 
 dropout_10 (Dropout)        (None, 5408)             

In [29]:
history = model.fit(x=train_set, batch_size=32, epochs=25, validation_data=valid_set)

Epoch 1/25
229/229 [==============================] - 86s 375ms/step - loss: 3.3468 - accuracy: 0.0692 - val_loss: 3.2147 - val_accuracy: 0.0733
Epoch 2/25
229/229 [==============================] - 86s 374ms/step - loss: 3.1736 - accuracy: 0.1147 - val_loss: 3.0174 - val_accuracy: 0.1200
Epoch 3/25
229/229 [==============================] - 87s 380ms/step - loss: 3.0465 - accuracy: 0.1454 - val_loss: 2.9723 - val_accuracy: 0.1533
Epoch 4/25
229/229 [==============================] - 85s 373ms/step - loss: 2.9518 - accuracy: 0.1692 - val_loss: 2.9465 - val_accuracy: 0.1733
Epoch 5/25
229/229 [==============================] - 87s 382ms/step - loss: 2.8770 - accuracy: 0.1897 - val_loss: 2.8733 - val_accuracy: 0.1333
Epoch 6/25
229/229 [==============================] - 85s 371ms/step - loss: 2.8012 - accuracy: 0.2129 - val_loss: 2.7871 - val_accuracy: 0.1667
Epoch 7/25
229/229 [==============================] - 87s 378ms/step - loss: 2.7242 - accuracy: 0.2311 - val_loss: 2.7778 - val_ac